# MusiCA

## Setup

In [7]:

using RemoteREPL
remoteREPLStarted::Bool = false
if !remoteREPLStarted
  @async serve_repl()
  remoteREPLStarted = true
end

true

#### Yhdistä REPL notebookkiin
`connect_repl()`

#### Hae muuttujia
`x = @remote(x)`  
`a, b = @remote(a,b)`

#### doc
https://c42f.github.io/RemoteREPL.jl/dev/

## Scratch

### using, new_state, ca & state

In [1]:
using Musica, BenchmarkTools, StaticArrays, Transducers, Plots, Folds
function new_state(::Type{Val{L}}) where {L}
  let bla = zeros(Bool, L)
    bla[1] = 1
    Row{2}(SizedVector{L}(bla))
  end
end

new_state(::Val{L}) where {L} = new_state(Val{L})

new_state(v::Integer) = new_state(Val(v))

ca = CA.Elementary{2,1}(110)
state=new_state(Val{32})
n_generations=20

caneuron = CANeuron{2,32}(ca, n_generations)

xf_printer(label) = Map() do x
  println(label, ": ", x)
  return x  # just return it as-is
end


xf_printer (generic function with 1 method)

In [9]:
Base.isinteractive()

false

In [10]:
using Transducers, Plots, Lazy, Musica, StaticArrays

# function new_state(row_len)
#   @SVector ones(Int, row_len)
# end

# include("./CA.jl")
steps = 100
res = Lazy.iterated(ca, new_state(160)) |> Take(steps) |> collect

heatmap(stack(res)',
  yflip=true,
  c=cgrad([:white, :black]),
  legend=:none,
  axis=false,
  ticks=false)

In [ ]:
using Lazy, Plots
steps=10
res = Lazy.iterated(CA.Elementaryy{2}(110), state) |> Map(Musica.count_ones) |> Take(steps) |> collect
plot(res)


### metaheur

#### opt

In [ ]:
using Plots, Musica, Transducers, StatsBase, Metaheuristics

function _plotopres(opres)
  @show extrema(fvals(opres))
  want_test_data = Musica._test_wanted_output_rastr(10, 0.19)
  _parser = Musica._test_parser_dynamic()
  _inp_gen = full_data_generator(want_test_data) #() -> (want_test_data, 1:length(want_test_data))
  # _res_gen = Musica.full_result_generator(want_test_data)
  # fff = Musica.to_obj_fn(_parser, _res_gen, Musica.make_fitness_function())

  _res_gen = input_based_result_gen()
  fff = create_obj_fn(_parser, _inp_gen, _res_gen, Musica.create_fitness_fn())

  best_sol_bits = get_best_at(fff, opres)
  # @show all(==(0), best_sol_bits)
  @show length(best_sol_bits)

  @show minimizer(opres) == best_sol_bits
  @show best_sol_bits
  @show Int(undigits(best_sol_bits[1:Musica._bits_per_stack_size()]))
  (_, cas) = best_sol_bits |> _parser
  @show cas
  for can in cas
    @show can
  end

  # (_want, _res) = _inp_gen()

  # best_sol_bits = minimizer(opres)
  (wanted, inp) = _inp_gen() |> maybe_collect
  result = _res_gen(inp, cas) |>
           Map(@<(normalize_num(2^16 - 1)) ∘ row_from_gray) |> maybe_collect

  @show extrema(result)
  @show StatsBase.rmsd(result, wanted)
  plot(result, label="result", seriescolor=:red)
  vline!([length(Musica._test_wanted_output_tan(π))], label="training data cutoff")
  plot!(wanted, label="target", seriescolor=:green)
end

_plotopres(opres)


In [ ]:
using Transducers, StaticArrays, Musica

#state=Musica.Row{2}(new_state(Val{16}))

test_ca = CA.Elementary{2}(110)
test_ca2 = CA.Elementary{2}(54)

trf=TeeRF(Map(test_ca)'(append!!)
       , Map(test_ca2)'(append!!))

foldl(trf, Map(identity), [state])
# transducereitten adjointit toimii foldl:n kanssa näin:
# Map(test_ca)'(append!!)([], test_state) ==
#   [1, 1, 0, 0, 0, 0, 0, 0]
# 
# eli ton palauttaman funktion vasen parametri on acc, oikea on varsinainen itemi.



In [ ]:
using Musica
let ca = ElementaryCA{2}(110), state = Row{2,5}(ones(Bool,5))
  specs=@which(ca(state)).specializations;
  foreach(specs) do spec
    @show spec
  end
end


In [ ]:
using BenchmarkTools, LoopVectorization
let n = 5_000

  function fill_vect(s)
    rand(Bool, s)
  end

  function fill_vect_for(s)
    v = Vector{Bool}(undef, s)
    @inbounds for i = 1:s
      v[i] = rand(Bool)
    end
    v
  end

  function fill_vect_for_simd(s)
    v = Vector{Bool}(undef, s)
    @inbounds @simd ivdep for i = 1:s
      v[i] = rand(Bool)
    end
    v
  end

  function fill_bitvect(s)
    BitVector(rand(Bool, s))
  end

  function fill_bitvect_for(s)
    v = BitVector(undef, s)
    @inbounds for i = 1:s
      v[i] = rand(Bool)
    end
    v
  end

  function fill_bitvect_for_simd(s)
    v = BitVector(undef, s)
    @inbounds @simd ivdep for i = 1:s
      v[i] = rand(Bool)
    end
    v
  end

  bset = @benchmarkset "ble" begin
    @case "fill_vect" $fill_vect($n)
    @case "fill_vect_for" $fill_vect_for($n)
    @case "fill_vect_for_simd" $fill_vect_for_simd($n)

    @case "fill_bitvect" $fill_bitvect($n)
    @case "fill_bitvect_for" $fill_bitvect_for($n)
    @case "fill_bitvect_for_simd" $fill_bitvect_for_simd($n)
  end

  for (setname, set) in BenchmarkTools.run(bset)
    for (name, trial) in set
        display(name)
        display(trial)
      end
  end

  # for (name, trial) in BenchmarkTools.run(bset)["ble"]
  #   display(name)
  #   display(trial)
  # end

end

In [ ]:
using Musica, Transducers, StaticArrays, StatsBase

indivs = SizedVector{10}(1:10 |> Map(x -> rand(Bool, rand(1:20))) |> collect);
st=GA.State{10}(indivs, GA.Options(objective_fn=x->0.0));
StatsBase.sample(GA.individuals(st), 2;replace=false)

In [ ]:
using Plots
histogram(randn(5000),seriescolor=:green)

In [2]:
module _typedispatch
using StaticArrays

drop(a, n) = _drop(SizedVector{length(a)}(a), Val{n})

_drop(a::SizedVector{0}, ::Type{Val{N}}) where N = a
_drop(a::SizedVector{Len}, ::Type{Val{0}}) where Len = a
_drop(a::SizedVector{Len}, ::Type{Val{N}}) where {Len,N} = _drop(SizedVector{Len - 1}(a[2:end]), Val{N - 1})

end

@show(_typedispatch.drop([1, 2, 3, 4, 5], 2))
@show(_typedispatch.drop([1, 2, 3, 4, 5], 6))

module _tailcalls

function drop(a, n)
  if isempty(a) || n == 0
    a
  else
    drop(a[2:end], n - 1)
  end
end

end

@show(_tailcalls.drop([1, 2, 3, 4, 5], 2))
@show(_tailcalls.drop([1, 2, 3, 4, 5], 6))

_typedispatch.drop([1, 2, 3, 4, 5], 2) = [3, 4, 5]
_typedispatch.drop([1, 2, 3, 4, 5], 6) = Int64[]


_tailcalls.drop([1, 2, 3, 4, 5], 2) = [3, 4, 5]
_tailcalls.drop([1, 2, 3, 4, 5], 6) = Int64[]


Int64[]

In [21]:
module _typeabuse
abstract type Call{Fn,Args,KW} end

@inline (c::Type{Call{Fn,Args,KW}})() where {Fn,Args,KW} = Fn(Args...; KW...)
@inline (c::Type{Call{Fn,Args}})() where {Fn,Args} = Fn(Args...)
@inline (c::Type{Call{Fn}})() where {Fn} = Fn()

end

let c = _typeabuse.Call{+,(1, 2)}
  @show c
  @show c()
end

let c = _typeabuse.Call{digits,5,(; base = 2)}
  println()
  @show c
  @show c()
end

let c = _typeabuse.Call{rand}
  println()
  @show c
  @show c()
end



c = Main._typeabuse.Call{+, (1, 2)}
c() = 3

c = Main._typeabuse.Call{digits, 5, (base = 2,)}
c() = [1, 0, 1]

c = Main._typeabuse.Call{rand}
c() = 0.35762394748720494


0.35762394748720494

In [31]:
using BenchmarkTools

module _traits
using WhereTraits

abstract type Abstract end
abstract type DomainType end

struct Discrete <: DomainType end

struct Continuous <: DomainType end

DomainType(_::Type{<:Abstract}) = Discrete()

struct ADiscreteType <: Abstract 
  value::Int
end

struct AContinuousType <: Abstract end
DomainType(::Type{AContinuousType}) = Continuous()

valueof(v::ADiscreteType) = v.value

@traits do_stuff_where(x::T) where {T<:Abstract, _traits.DomainType(T)::_traits.Discrete} = x |> valueof
@traits do_stuff_where(x::T) where {T<:Abstract} = 0
@traits do_stuff_where(_) = 0

do_stuff_trait(x::Abstract) = _do_stuff_trait(DomainType(x|>typeof), x)
_do_stuff_trait(::Discrete, x) = x |> valueof
end

let it=_traits.ADiscreteType(666)
  bset = @benchmarkset "traits" begin
    @case "WhereTraits discrete" _traits.do_stuff_where($it)
    @case "vanilla traits" _traits.do_stuff_trait($it)
  end
  
  for (setname, set) in BenchmarkTools.run(bset)
    for (name, trial) in set
        display(name)
        display(trial)
      end
  end
end


"WhereTraits discrete"

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   0.001 ns …  7.709 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     83.000 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   74.854 ns ± 79.075 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                                            █  
  ▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▂
  0.001 ns        Histogram: frequency by time          84 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

"vanilla traits"

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   0.001 ns …  3.667 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      0.001 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   16.103 ns ± 41.757 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                            
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▇ ▂
  0.001 ns        Histogram: frequency by time          42 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

┌ Warning: Replacing docs for `Main._traits.do_stuff_where :: Tuple{T1} where T1<:Main._traits.Abstract` in module `Main._traits`
└ @ Base.Docs docs/Docs.jl:243


In [ ]:
module _rules
using Metatheory

const r1 = @rule #sin(2(~x)) --> 2sin(~x)*cos(~x)

# const expr = :(sin(2z))
end

_rules